# Introduction

In this notebook, I explore outliers in 10 variables of interest in the College Scorecard dataset based on the relative probabilities of their values. I present the 20 most unlikely values for each variable along with the corresponding school ID code.

# Setup

In [13]:
def powerset_gen(sequence):
    l=[]
    if len(sequence)<=1:
        yield sequence
        yield []
    else:
        for item in powerset(sequence[1:]):
            yield [sequence[0]]+item
            yield item
            
def powerset(seq):
    l=[]
    gen=powerset_gen(seq)
    for item in gen:
        l.append(item)
    return l

In [11]:
seq = [1,5,9]

In [15]:
powerset(seq)

[[1, 5, 9], [5, 9], [1, 9], [9], [1, 5], [5], [1], []]

In [40]:
import bayeslite
from bayeslite.read_pandas import bayesdb_read_pandas_df
import bdbcontrib
from bdbcontrib import cursor_to_df as df
from bdbcontrib.recipes import quickstart
import pandas as pd
import numpy
import re
import matplotlib
from matplotlib import ft2font
import matplotlib.pyplot as plt
%matplotlib inline
from bdbcontrib.recipes import quickstart
import crosscat
import crosscat.MultiprocessingEngine as ccme
import bayeslite.metamodels.crosscat
import os
import pandas as pd
import crosscat
from bdbcontrib.recipes import quickstart
import bayeslite
from bayeslite.read_pandas import bayesdb_read_pandas_df

In [25]:
def powerset_gen(seq):
    if len(seq)<=1:
        yield seq
        yield []
        
    else:
        for item in powerset_gen(seq[1:]):
            yield [seq[0]]+item
            yield item
            
def powerset(seq):
    l=[]
    for item in powerset_gen(seq):
        l.append(item)
    return l

In [26]:
sequ = [5,4,6,2]

In [27]:
powerset(sequ)

[[5, 4, 6, 2],
 [4, 6, 2],
 [5, 6, 2],
 [6, 2],
 [5, 4, 2],
 [4, 2],
 [5, 2],
 [2],
 [5, 4, 6],
 [4, 6],
 [5, 6],
 [6],
 [5, 4],
 [4],
 [5],
 []]

In [42]:
#Load the dataset via pandas
DATA = pd.read_csv("Most+Recent+Cohorts+(All+Data+Elements).csv", delimiter=',', low_memory = False)
DATA = DATA.replace('PrivacySuppressed', '-1')

#Restricting the dataframe to 10 variables of interest
df10=DATA.loc[:,['OPEID','UGDS','ADM_RATE_ALL','TUITIONFEE_PROG','DEP_DEBT_MDN','LO_INC_DEATH_YR2_R','MD_INC_DEATH_YR2_R','HI_INC_DEATH_YR2_R','AVGFACSAL','md_faminc','famine']]

#Renaming the variables names by their definition
df10.columns = ['OPEID_of_institution','Enrollment_of_undergraduate_degree_seeking_students',
'Admission_rate_for_all_campuses_rolled_up_to_the_6_digit_OPE_ID',
'TUITIONFEE_PROG',
'The_median_debt_for_dependent_students',
'Percent_of_low_income_students_who_died_within_2_years',
'Percent_of_middle_income_students_who_died_within_2_years',
'Percent_of_high_income_students_who_died_within_2_years',
'Average_faculty_salary',
'Median_family_income',
'Average_family_income']

In [43]:
# Open an empty bdb file
bdb = bayeslite.bayesdb_open("chris.bdb")

# Link your dataframe to your bdb file
bayesdb_read_pandas_df(bdb, "df3", df10, create=True)

data = quickstart(name='df3', bdb_path='chris.bdb')
q = data.q

BQL [SELECT COUNT(*) FROM "df3"] [()]
BQL [
        CREATE GENERATOR "df3_cc" IF NOT EXISTS FOR "df3" USING crosscat( GUESS(*) )] [()]


In [64]:
data.analyze(models = 25, iterations = 20)

BQL [INITIALIZE 25 MODELS IF NOT EXISTS FOR df3_cc] [()]
BQL [ANALYZE df3_cc FOR 20 ITERATIONS CHECKPOINT 1 ITERATION WAIT] [()]
BQL [SELECT iterations FROM bayesdb_generator_model
                           WHERE generator_id = (
                            SELECT id FROM bayesdb_generator WHERE name = ?)] [('df3_cc',)]


,count of models
iterations,
20,25


# Enrollment

In [65]:
q('''
CREATE TEMP TABLE unlikely_enrollment AS
    ESTIMATE OPEID_of_institution, Enrollment_of_undergraduate_degree_seeking_students,
        PREDICTIVE PROBABILITY OF Enrollment_of_undergraduate_degree_seeking_students
            AS "Relative Probability of Enrollment"
    FROM df2_cc;
''')


BQL [
CREATE TEMP TABLE unlikely_enrollment AS
    ESTIMATE OPEID_of_institution, Enrollment_of_undergraduate_degree_seeking_students,
        PREDICTIVE PROBABILITY OF Enrollment_of_undergraduate_degree_seeking_students
            AS "Relative Probability of Enrollment"
    FROM df2_cc;
] [()]


""


In [66]:
q('''
SELECT * FROM unlikely_enrollment
    WHERE Enrollment_of_undergraduate_degree_seeking_students IS NOT NULL
    ORDER BY "Relative Probability of Enrollment" ASC LIMIT 20;
''')

BQL [
SELECT * FROM unlikely_enrollment
    WHERE Enrollment_of_undergraduate_degree_seeking_students IS NOT NULL
    ORDER BY "Relative Probability of Enrollment" ASC LIMIT 20;
] [()]


,OPEID_of_institution,Enrollment_of_undergraduate_degree_seeking_students,Relative Probability of Enrollment
0,4118700,1819,0.000057
1,3210300,1739,0.000109
2,3284300,1642,0.000112
3,3106000,1648,0.000113
4,129400,17611,0.000118
5,3956317,1713,0.000119
6,2156600,1557,0.000120
7,262900,33427,0.000120
8,462500,17467,0.000121
9,3070500,1550,0.000122


"04118700"	American College of Technology
???
"03284300"	Michigan Jewish Institute

# Admission Rate

In [67]:
q('''
CREATE TEMP TABLE unlikely_admission_rate AS
    ESTIMATE OPEID_of_institution, Admission_rate_for_all_campuses_rolled_up_to_the_6_digit_OPE_ID,
        PREDICTIVE PROBABILITY OF Admission_rate_for_all_campuses_rolled_up_to_the_6_digit_OPE_ID
            AS "Relative Probability of Admission Rate"
    FROM df2_cc;
''')

BQL [
CREATE TEMP TABLE unlikely_admission_rate AS
    ESTIMATE OPEID_of_institution, Admission_rate_for_all_campuses_rolled_up_to_the_6_digit_OPE_ID,
        PREDICTIVE PROBABILITY OF Admission_rate_for_all_campuses_rolled_up_to_the_6_digit_OPE_ID
            AS "Relative Probability of Admission Rate"
    FROM df2_cc;
] [()]


""


In [68]:
q('''
SELECT * FROM unlikely_admission_rate
    WHERE Admission_rate_for_all_campuses_rolled_up_to_the_6_digit_OPE_ID IS NOT NULL
    ORDER BY "Relative Probability of Admission Rate" ASC LIMIT 20;
''')

BQL [
SELECT * FROM unlikely_admission_rate
    WHERE Admission_rate_for_all_campuses_rolled_up_to_the_6_digit_OPE_ID IS NOT NULL
    ORDER BY "Relative Probability of Admission Rate" ASC LIMIT 20;
] [()]


,OPEID_of_institution,Admission_rate_for_all_campuses_rolled_up_to_the_6_digit_OPE_ID,Relative Probability of Admission Rate
0,3014900,0.0000,0.140250
1,174600,0.2100,0.247878
2,622500,0.0000,0.248284
3,243300,0.1386,0.266669
4,340900,0.2731,0.269580
5,133900,0.3225,0.280063
6,242400,0.2272,0.281793
7,236600,0.2143,0.306358
8,2340500,0.3158,0.318811
9,679100,0.3319,0.319487


# Tuition

In [69]:
q('''
CREATE TEMP TABLE unlikely_tuition AS
    ESTIMATE OPEID_of_institution, TUITIONFEE_PROG,
        PREDICTIVE PROBABILITY OF TUITIONFEE_PROG
            AS "Relative Probability of Tuition"
    FROM df2_cc;
''')

BQL [
CREATE TEMP TABLE unlikely_tuition AS
    ESTIMATE OPEID_of_institution, TUITIONFEE_PROG,
        PREDICTIVE PROBABILITY OF TUITIONFEE_PROG
            AS "Relative Probability of Tuition"
    FROM df2_cc;
] [()]


""


In [70]:
q('''
SELECT * FROM unlikely_tuition
    WHERE TUITIONFEE_PROG IS NOT NULL
    ORDER BY "Relative Probability of Tuition" ASC LIMIT 20;
''')

BQL [
SELECT * FROM unlikely_tuition
    WHERE TUITIONFEE_PROG IS NOT NULL
    ORDER BY "Relative Probability of Tuition" ASC LIMIT 20;
] [()]


,OPEID_of_institution,TUITIONFEE_PROG,Relative Probability of Tuition
0,2600001,1260,0.000006
1,4166000,760,0.000006
2,3160300,1320,0.000006
3,2560700,539,0.000006
4,920408,1881,0.000007
5,3944400,999,0.000007
6,4004300,1150,0.000007
7,3736300,501,0.000007
8,3067400,2280,0.000008
9,2250900,1350,0.000008


# Median Debt

In [45]:
q('''
CREATE TEMP TABLE unlikely_median_debt AS
    ESTIMATE OPEID_of_institution, The_median_debt_for_dependent_students,
        PREDICTIVE PROBABILITY OF The_median_debt_for_dependent_students
            AS "Relative Probability of Median Debt"
    FROM df2_cc;
''')

BQL [
CREATE TEMP TABLE unlikely_median_debt AS
    ESTIMATE OPEID_of_institution, The_median_debt_for_dependent_students,
        PREDICTIVE PROBABILITY OF The_median_debt_for_dependent_students
            AS "Relative Probability of Median Debt"
    FROM df2_cc;
] [()]


""


In [46]:
q('''
SELECT * FROM unlikely_median_debt
    WHERE The_median_debt_for_dependent_students IS NOT NULL
    ORDER BY "Relative Probability of Median Debt" ASC LIMIT 20;
''')

BQL [
SELECT * FROM unlikely_median_debt
    WHERE The_median_debt_for_dependent_students IS NOT NULL
    ORDER BY "Relative Probability of Median Debt" ASC LIMIT 20;
] [()]


,OPEID_of_institution,The_median_debt_for_dependent_students,Relative Probability of Median Debt
0,329300,27000.0,0.000018
1,258700,27000.0,0.000022
2,335300,27000.0,0.000022
3,326200,26948.0,0.000023
4,341100,26777.5,0.000027
5,396600,28999.0,0.000029
6,185000,27000.0,0.000030
7,112300,28000.0,0.000032
8,219900,27000.0,0.000033
9,187400,26942.5,0.000033


# Percentage of Students from Low-Income Families who Died within 2 Years

In [71]:
q('''
CREATE TEMP TABLE unlikely_low_income_student_deaths AS
    ESTIMATE OPEID_of_institution, 'Percent_of_low_income_students_who_died_within_2_years',
        PREDICTIVE PROBABILITY OF Percent_of_low_income_students_who_died_within_2_years
            AS "Relative Probability of Percentage Student Deaths from Low Income Families"
    FROM df2_cc;
''')

BQL [
CREATE TEMP TABLE unlikely_low_income_student_deaths AS
    ESTIMATE OPEID_of_institution, 'Percent_of_low_income_students_who_died_within_2_years',
        PREDICTIVE PROBABILITY OF Percent_of_low_income_students_who_died_within_2_years
            AS "Relative Probability of Percentage Student Deaths from Low Income Families"
    FROM df2_cc;
] [()]


ERROR: 
Traceback (most recent call last):

  File "/home/christina/bdbcontrib/build/lib.linux-x86_64-2.7/bdbcontrib/recipes.py", line 149, in query
    res = self.bdb.execute(query_string, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bayesdb.py", line 215, in execute
    self.tracer, self._do_execute, string, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bayesdb.py", line 223, in _maybe_trace
    return meth(string, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bayesdb.py", line 264, in _do_execute
    cursor = bql.execute_phrase(self, phrase, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bql.py", line 83, in execute_phrase
    compiler.compile_query(bdb, phrase.query, out)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/compiler.py", line 213, in compile_query
    compile_estimate(bdb, query, out)

  File "/home/chris

In [ ]:
q('''
SELECT * FROM unlikely_low_income_student_deaths
    WHERE Percent_of_low_income_students_who_died_within_2_years IS NOT NULL
    ORDER BY "Relative Probability of Percentage Student Deaths from Low Income Families" ASC LIMIT 20;
''')

# Percentage of Students from Middle-Income Families who Died within 2 Years

In [72]:
q('''
CREATE TEMP TABLE unlikely_middle_income_student_deaths AS
    ESTIMATE OPEID_of_institution, Percent_of_middle_income_students_who_died_within_2_years,
        PREDICTIVE PROBABILITY OF Percent_of_middle_income_students_who_died_within_2_years
            AS "Relative Probability of Percentage Student Deaths from Middle Income Families"
    FROM df2_cc;
''')

BQL [
CREATE TEMP TABLE unlikely_middle_income_student_deaths AS
    ESTIMATE OPEID_of_institution, Percent_of_middle_income_students_who_died_within_2_years,
        PREDICTIVE PROBABILITY OF Percent_of_middle_income_students_who_died_within_2_years
            AS "Relative Probability of Percentage Student Deaths from Middle Income Families"
    FROM df2_cc;
] [()]


ERROR: 
Traceback (most recent call last):

  File "/home/christina/bdbcontrib/build/lib.linux-x86_64-2.7/bdbcontrib/recipes.py", line 149, in query
    res = self.bdb.execute(query_string, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bayesdb.py", line 215, in execute
    self.tracer, self._do_execute, string, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bayesdb.py", line 223, in _maybe_trace
    return meth(string, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bayesdb.py", line 264, in _do_execute
    cursor = bql.execute_phrase(self, phrase, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bql.py", line 83, in execute_phrase
    compiler.compile_query(bdb, phrase.query, out)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/compiler.py", line 213, in compile_query
    compile_estimate(bdb, query, out)

  File "/home/chris

In [ ]:
q('''
SELECT * FROM unlikely_middle_income_student_deaths
    WHERE Percent_of_middle_income_students_who_died_within_2_years IS NOT NULL
    ORDER BY "Relative Probability of Percentage Student Deaths from Middle Income Families" ASC LIMIT 20;
''')

# Percentage of Students from Middle-Income Families who Died within 2 Years

In [73]:
q('''
CREATE TEMP TABLE unlikely_high_income_student_deaths AS
    ESTIMATE OPEID_of_institution, Percent_of_high_income_students_who_died_within_2_years,
        PREDICTIVE PROBABILITY OF Percent_of_high_income_students_who_died_within_2_years
            AS "Relative Probability of Percentage Student Deaths from High Income Families"
    FROM df2_cc;
''')

BQL [
CREATE TEMP TABLE unlikely_high_income_student_deaths AS
    ESTIMATE OPEID_of_institution, Percent_of_high_income_students_who_died_within_2_years,
        PREDICTIVE PROBABILITY OF Percent_of_high_income_students_who_died_within_2_years
            AS "Relative Probability of Percentage Student Deaths from High Income Families"
    FROM df2_cc;
] [()]


ERROR: 
Traceback (most recent call last):

  File "/home/christina/bdbcontrib/build/lib.linux-x86_64-2.7/bdbcontrib/recipes.py", line 149, in query
    res = self.bdb.execute(query_string, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bayesdb.py", line 215, in execute
    self.tracer, self._do_execute, string, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bayesdb.py", line 223, in _maybe_trace
    return meth(string, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bayesdb.py", line 264, in _do_execute
    cursor = bql.execute_phrase(self, phrase, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bql.py", line 83, in execute_phrase
    compiler.compile_query(bdb, phrase.query, out)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/compiler.py", line 213, in compile_query
    compile_estimate(bdb, query, out)

  File "/home/chris

In [58]:
q('''
SELECT * FROM unlikely_high_income_student_deaths
    WHERE Percent_of_high_income_students_who_died_within_2_years IS NOT NULL
    ORDER BY "Relative Probability of Percentage Student Deaths from High Income Families" ASC LIMIT 20;
''')

ERROR: 
Traceback (most recent call last):

  File "/home/christina/bdbcontrib/build/lib.linux-x86_64-2.7/bdbcontrib/recipes.py", line 149, in query
    res = self.bdb.execute(query_string, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bayesdb.py", line 215, in execute
    self.tracer, self._do_execute, string, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bayesdb.py", line 223, in _maybe_trace
    return meth(string, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bayesdb.py", line 264, in _do_execute
    cursor = bql.execute_phrase(self, phrase, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bql.py", line 61, in execute_phrase
    out.getbindings())

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bql.py", line 648, in execute_wound
    return bdb.sql_execute(sql, bindings)

  File "/home/christina/bayeslite/build/lib.linu

BQL [
SELECT * FROM unlikely_high_income_student_deaths
    WHERE Percent_of_high_income_more_than_75k_in_nominal_family_income_students_who_died_within_2_years IS NOT NULL
    ORDER BY "Relative Probability of Percentage Student Deaths from High Income Families" ASC LIMIT 20;
] [()]


# Average Faculty Salary

In [52]:
q('''
CREATE TEMP TABLE unlikely_faculty_salary AS
    ESTIMATE OPEID_of_institution, Average_faculty_salary,
        PREDICTIVE PROBABILITY OF Average_faculty_salary
            AS "Relative Probability of Average Faculty Salary"
    FROM df2_cc;
''')

BQL [
CREATE TEMP TABLE unlikely_faculty_salary AS
    ESTIMATE OPEID_of_institution, Average_faculty_salary,
        PREDICTIVE PROBABILITY OF Average_faculty_salary
            AS "Relative Probability of Average Faculty Salary"
    FROM df2_cc;
] [()]


""


##### q('''
SELECT * FROM unlikely_faculty_salary
    WHERE Average_faculty_salary IS NOT NULL
    ORDER BY "Relative Probability of Average Faculty Salary" ASC LIMIT 20;
''')

# Median Family Income

In [54]:
q('''
CREATE TEMP TABLE unlikely_median_family_income AS
    ESTIMATE OPEID_of_institution, Median_family_income,
        PREDICTIVE PROBABILITY OF Median_family_income
            AS "Relative Probability of Median Family Income"
    FROM df2_cc;
''')

BQL [
CREATE TEMP TABLE unlikely_median_family_income AS
    ESTIMATE OPEID_of_institution, Median_family_income,
        PREDICTIVE PROBABILITY OF Median_family_income
            AS "Relative Probability of Median Family Income"
    FROM df2_cc;
] [()]


""


In [55]:
q('''
SELECT * FROM unlikely_median_family_income
    WHERE Median_family_income IS NOT NULL
    ORDER BY "Relative Probability of Median Family Income" ASC LIMIT 20;
''')

BQL [
SELECT * FROM unlikely_median_family_income
    WHERE Median_family_income IS NOT NULL
    ORDER BY "Relative Probability of Median Family Income" ASC LIMIT 20;
] [()]


,OPEID_of_institution,Median_family_income,Relative Probability of Median Family Income
0,138500,116850.0,9.141587e-07
1,212000,108518.0,1.820038e-06
2,338800,112696.0,1.828097e-06
3,330400,110473.0,1.972327e-06
4,140200,108100.0,2.102007e-06
5,336700,109701.5,2.164920e-06
6,209500,103581.5,2.439786e-06
7,214800,109499.5,2.543277e-06
8,207800,112164.0,2.558729e-06
9,2532600,90146.0,2.782879e-06


# Average Family Income 

In [74]:
q('''
CREATE TEMP TABLE unlikely_avg_family_income AS
    ESTIMATE OPEID_of_institution, Average_family_income,
        PREDICTIVE PROBABILITY OF Average_family_income
            AS "Relative Probability of Average Family Income"
    FROM df2_cc;
''')

BQL [
CREATE TEMP TABLE unlikely_avg_family_income AS
    ESTIMATE OPEID_of_institution, Average_family_income,
        PREDICTIVE PROBABILITY OF Average_family_income
            AS "Relative Probability of Average Family Income"
    FROM df2_cc;
] [()]


ERROR: 
Traceback (most recent call last):

  File "/home/christina/bdbcontrib/build/lib.linux-x86_64-2.7/bdbcontrib/recipes.py", line 149, in query
    res = self.bdb.execute(query_string, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bayesdb.py", line 215, in execute
    self.tracer, self._do_execute, string, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bayesdb.py", line 223, in _maybe_trace
    return meth(string, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bayesdb.py", line 264, in _do_execute
    cursor = bql.execute_phrase(self, phrase, bindings)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/bql.py", line 83, in execute_phrase
    compiler.compile_query(bdb, phrase.query, out)

  File "/home/christina/bayeslite/build/lib.linux-x86_64-2.7/bayeslite/compiler.py", line 213, in compile_query
    compile_estimate(bdb, query, out)

  File "/home/chris

In [ ]:
q('''
SELECT * FROM unlikely_avg_family_income
    WHERE Average_family_income IS NOT NULL
    ORDER BY "Relative Probability of Average Family Income" ASC LIMIT 20;
''')